In [ ]:
import pandas as pd
import numpy as np
import scipy as sp
from sklearn.metrics.pairwise import cosine_similarity
import operator
%matplotlib inline

In [ ]:
anime = pd.read_csv('../input/anime-recommendations-database/anime.csv')
rating = pd.read_csv('../input/anime-recommendations-database/rating.csv')

In [ ]:
anime.head()

In [ ]:
rating.head()

평가를 내리지 않은 값 "-1"은 NaN으로 대체

In [ ]:
rating.rating.replace(-1,np.nan, regex=True, inplace = True)
rating.head()

In [ ]:
해당 추천엔진은 TV판 애니메이션만 대상으로 함.

In [ ]:
anime_tv = anime[anime['type']=='TV']
anime_tv.head()

anime_id 컬럼을 기준으로 조인

In [ ]:
merged = rating.merge(anime_tv, left_on = 'anime_id', right_on = 'anime_id', suffixes= ['_user', ''])
merged.rename(columns = {'rating_user':'user_rating'}, inplace = True)

In [ ]:
merged.head()

퍼포먼스 문제로 데이터 프레임 길이를 10,000명의 사용자로 제한.

In [ ]:
merged=merged[['user_id', 'name', 'user_rating']]
merged_sub= merged[merged.user_id <= 10000]
merged_sub.head()


CF를 위해 피벗 테이블을 만들고 각 축은 유저와 애니메이션으로 구성.

In [ ]:
piv = merged_sub.pivot_table(index=['user_id'], columns=['name'], values='user_rating')

In [ ]:
print(piv.shape)
piv

In [ ]:
# Note: 표준화 할 각 등급에서 평균을 뺀 값
# 하나의 평가 만 있거나 모든 것을 동일하게 평가 한 모든 사용자는 삭제

# 값을 정규화
piv_norm = piv.apply(lambda x: (x-np.mean(x))/(np.max(x)-np.min(x)), axis=1)
piv_norm

In [ ]:

# 평가하지 않은 사용자를 나타내는 0 만 포함하는 모든 열을 삭제
piv_norm.fillna(0, inplace=True)
piv_norm = piv_norm.T
piv_norm = piv_norm.loc[:, (piv_norm != 0).any(axis=0)]

piv_norm

In [ ]:

#다음 함수에서 데이터를 읽으려면 희소 행렬 형식이어야함.
piv_sparse = sp.sparse.csr_matrix(piv_norm.values)
piv_sparse

이 행렬은 각 사용자 / 사용자 배열 쌍과 항목 / 항목 배열 쌍 사이의 계산 된 코사인 유사성 값을 보여줌

In [ ]:
item_similarity = cosine_similarity(piv_sparse)
user_similarity = cosine_similarity(piv_sparse.T)


In [ ]:
print(item_similarity.shape)
item_similarity

In [ ]:
print(user_similarity.shape)
user_similarity

In [ ]:
# 데이터 프레임 객체에 유사성 요소 삽입

item_sim_df = pd.DataFrame(item_similarity, index = piv_norm.index, columns = piv_norm.index)
user_sim_df = pd.DataFrame(user_similarity, index = piv_norm.columns, columns = piv_norm.columns)

In [ ]:
item_sim_df.head()

In [ ]:
user_sim_df.head()

In [ ]:
# 이 함수는 코사인 유사성 값이 가장 높은 상위 10 개의 애니메이션을 반환.

def top_animes(anime_name):
    count = 1
    print('Similar shows to {} include:\n'.format(anime_name))
    for item in item_sim_df.sort_values(by = anime_name, ascending = False).index[1:11]:
        print('No. {}: {}'.format(count, item))
        count +=1  

In [ ]:
# 이 함수는 가장 높은 유사성 값을 가진 상위 5 명의 사용자를 반환. 

def top_users(user):
    
    if user not in piv_norm.columns:
        return('No data available on user {}'.format(user))
    
    print('Most Similar Users:\n')
    sim_values = user_sim_df.sort_values(by=user, ascending=False).loc[:,user].tolist()[1:11]
    sim_users = user_sim_df.sort_values(by=user, ascending=False).index[1:11]
    zipped = zip(sim_users, sim_values,)
    for user, sim in zipped:
        print('User #{0}, Similarity value: {1:.2f}'.format(user, sim)) 

In [ ]:
# 이 함수는 유사한 사용자 당 가장 높은 등급의 애니를 포함하는 목록을 구성
# 목록에 표시되는 빈도와 함께 애니메이션 제목을 반환

def similar_user_recs(user):
    
    if user not in piv_norm.columns:
        return('No data available on user {}'.format(user))
    
    sim_users = user_sim_df.sort_values(by=user, ascending=False).index[1:11]
    best = []
    most_common = {}
    
    for i in sim_users:
        max_score = piv_norm.loc[:, i].max()
        best.append(piv_norm[piv_norm.loc[:, i]==max_score].index.tolist())
    for i in range(len(best)):
        for j in best[i]:
            if j in most_common:
                most_common[j] += 1
            else:
                most_common[j] = 1
    sorted_list = sorted(most_common.items(), key=operator.itemgetter(1), reverse=True)
    return sorted_list[:5]    

In [ ]:
# 이 함수는 유사한 사용자의 가중 평균을 계산
# 입력 사용자의 평가 예상을 결정하고 표시

def predicted_rating(anime_name, user):
    sim_users = user_sim_df.sort_values(by=user, ascending=False).index[1:1000]
    user_values = user_sim_df.sort_values(by=user, ascending=False).loc[:,user].tolist()[1:1000]
    rating_list = []
    weight_list = []
    for j, i in enumerate(sim_users):
        rating = piv.loc[i, anime_name]
        similarity = user_values[j]
        if np.isnan(rating):
            continue
        elif not np.isnan(rating):
            rating_list.append(rating*similarity)
            weight_list.append(similarity)
    return sum(rating_list)/sum(weight_list)  

In [ ]:
top_animes('Cowboy Bebop')

In [ ]:
top_users(3)

In [ ]:
similar_user_recs(3)

In [ ]:
predicted_rating('Cowboy Bebop', 3)

In [ ]:
# 유저 "3"이 시청 한 모든 애니의 목록을 만듬.

watched = piv.T[piv.loc[3,:]>0].index.tolist()
watched

In [ ]:
# 실제 값과 예측값 사이의 제곱 오차 목록을 만듬

errors = []
for i in watched:
    actual=piv.loc[3, i]
    predicted = predicted_rating(i, 3)
    errors.append((actual-predicted)**2)

In [ ]:
# 유저 3의 평균 제곱 오차.
np.mean(errors)

In [ ]:
piv.head()